/* Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

This is Tier 2 of the case study, which means that there'll be less guidance for you about how to setup
your local SQLite connection in PART 2 of the case study. This will make the case study more challenging for you: 
you might need to do some digging, aand revise the Working with Relational Databases in Python chapter in the previous resource.

Otherwise, the questions in the case study are exactly the same as with Tier 1. 

PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser, and
using the following Username and Password:

URL: https://sql.springboard.com/
Username: student
Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table,
    ii) the "Facilities" table, and
    iii) the "Members" table.

In this case study, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables. */

/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

In [ ]:
SELECT name, membercost 
FROM Facilities
WHERE membercost > 0;

/* Q2: How many facilities do not charge a fee to members? */

In [ ]:
SELECT name, membercost 
FROM Facilities
WHERE membercost = 0;

#A2: 4 facilities do not charge fees to members.

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */

In [ ]:
SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost > 0 and membercost < 0.2 * monthlymaintenance;

/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

In [ ]:
SELECT *
FROM Facilities
WHERE facid in (1,5);

/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

In [ ]:
SELECT
    name, 
    monthlymaintenance, 
    CASE WHEN monthlymaintenance >100 THEN 'Expensive'
    ELSE 'Cheap' END AS label
FROM Facilities as f

#A5: The name and monthly maintenance costs of the 4 facilities w/ m.cost >$100 are:
# Tennis Court 1, 200
# Tennis Court 2, 200
# Massage Room 1, 3000
# Massage Room 2, 3000


/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

In [ ]:
#This first answer returns the first & last names of members in descending order by joindate.
SELECT firstname, surname, joindate
FROM Members
ORDER BY joindate DESC

#This second answer returns the frist & last names of the member/s who joined in the latest joindate.
SELECT firstname, surname, joindate
FROM Members
INNER JOIN (
    SELECT MAX(joindate) as LatestDate
    FROM Members
    ) as m
ON Members.joindate = m.LatestDate

#A6: the last member is Darren Smith.

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

In [ ]:
SELECT DISTINCT f.name as facility_name, m.member_name
FROM Bookings as b
LEFT JOIN Facilities as f
ON b.facid = f.facid
LEFT JOIN (
    SELECT memid, concat(firstname, SPACE(1), surname) as member_name
    FROM Members) as m
ON b.memid = m.memid
WHERE b.facid in (0,1)
ORDER BY m.member_name, f.name

/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

In [ ]:
SELECT
    f.name as facility_name, 
    concat(m.firstname, SPACE(1), m.surname) as member_name, 
    (CASE WHEN b.memid = 0 AND f.guestcost*b.slots > 30 THEN f.guestcost*b.slots
     WHEN b.memid > 0 AND f.membercost*b.slots >30 THEN f.membercost*b.slots 
     END) as cost
FROM Bookings as b
LEFT JOIN Facilities as f
ON b.facid = f.facid
LEFT JOIN Members as m
ON b.memid = m.memid
WHERE starttime LIKE '2012-09-14%' AND 
    CASE WHEN b.memid = 0 AND f.guestcost*b.slots > 30 THEN f.guestcost*b.slots
     WHEN b.memid > 0 AND f.membercost*b.slots >30 THEN f.membercost*b.slots 
     END IS NOT NULL
ORDER BY cost DESC

/* Q9: This time, produce the same result as in Q8, but using a subquery. */

In [ ]:
SELECT facility_name, member_name, cost
FROM
    (SELECT
        (SELECT name
         FROM Facilities as f
         WHERE f.facid = b.facid) 
            as facility_name, 
        (SELECT
         concat(m.firstname, SPACE(1), m.surname) 
         FROM Members as m
         WHERE m.memid = b.memid)
            as member_name, 
        (CASE WHEN b.memid = 0 THEN f.guestcost*b.slots
         WHEN b.memid > 0 THEN f.membercost*b.slots 
         END) as cost
        FROM Bookings as b
        LEFT JOIN Facilities as f
        ON b.facid = f.facid
        WHERE starttime LIKE '2012-09-14%'
    ) as s
WHERE cost > 30
ORDER BY cost DESC

/* PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

In [1]:
import sqlite3
from sqlalchemy import create_engine
import pandas as pd

In [ ]:
# HereI try to run use "create_engine" like in the Datacamp resource on my .sql file, 
# but I get the error "DatabaseError: (sqlite3.DatabaseError) file is not a database
# [SQL: SELECT * FROM Facilities]
# Background on this error at: http://sqlalche.me/e/13/4xp6"

engine = create_engine('sqlite:///country_club_sqlite.sql')
with engine.connect() as con:
    rs = con.execute("SELECT * FROM Facilities")
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()


In [ ]:
# In my second attempt, I try to read my sql file by using this code I found in stack overflow. I get a syntax error.
# But even when I remove the code with the error, and run just the first 3 lines, 
# I get "sqlFile" as a str, and I still cannot work with it.



# Open and read the file as a single buffer
fd = open('country_club_sqlite.sql', 'r')
sqlFile = fd.read()
fd.close()

# all SQL commands (split on ';')
sqlCommands = sqlFile.split(';')

# Execute every command from the input file
for command in sqlCommands:
    # This will skip and report errors
    # For example, if the tables do not yet exist, this will skip over
    # the DROP TABLE commands
    try:
       c.execute(command)
    except OperationalError, msg:
       print "Command skipped: ", msg

In [ ]:
type(sqlFile)

In [ ]:
#In my 3rd and final attempt, I to use pd.read_sql_query(), but I realized I don't know how to set up a database,
# so I don't know how to connect to the database...

query = open('country_club_sqlite.sql', 'r')
# connection == the connection to your database
df = pd.read_sql_query(query.read(), connection)
query.close() 

In [2]:
# At this point, I gave up and decided to use the instructions and files
# provided by Springboard for Tier 1 work. This is copied and run below.

import os
os.getcwd()
path = 'C:\\Users\\grace\\Documents\\Data Science Bootcamp\\DSC-projects\\Springboard\\SQLFiles Tier 1'
os.chdir(path)

In [3]:
os.getcwd()

'C:\\Users\\grace\\Documents\\Data Science Bootcamp\\DSC-projects\\Springboard\\SQLFiles Tier 1'

In [4]:
# import sqlite3
from sqlite3 import Error
 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


QUESTIONS:
/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [8]:
#Note in the query, if I don't specify the date, then all facilities make more than 1000 USD.
#By restricting it to datetime to 2012-09-14 as in Q8 and Q9, 
#then we get 8 facilities that made less than 1000.

def select_all_tasks(conn):
    cur = conn.cursor()
    query1 = """
SELECT facility_name, total_revenue
FROM (
        SELECT
            f.name as facility_name, 
            (SUM(f.guestcost*b.slots) + SUM(f.membercost*b.slots)) as total_revenue
        FROM Bookings as b
        LEFT JOIN Facilities as f
        ON b.facid = f.facid
        LEFT JOIN Members as m
        ON b.memid = m.memid
        WHERE starttime LIKE '2012-09-14%'AND 
        CASE WHEN b.memid = 0 THEN f.guestcost*b.slots
        WHEN b.memid > 0 THEN f.membercost*b.slots END IS NOT NULL
        GROUP BY b.facid
        ORDER BY total_revenue DESC
    ) as s
WHERE total_revenue < 1000
        """
    cur.execute(query1)
    rows = cur.fetchall()
    for row in rows:
        print(row)

if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Tennis Court 2', 630)
('Tennis Court 1', 630)
('Massage Room 2', 539.4)
('Squash Court', 336.0)
('Badminton Court', 232.5)
('Pool Table', 105)
('Snooker Table', 50)
('Table Tennis', 40)


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [10]:
#This returns two members in (surname, firstname) order. First the member, and then who recommended them.

def select_all_tasks(conn):
    cur = conn.cursor()
    query1 = """
SELECT m1.surname, m1.firstname, m2.surname as rec_surname, m2.firstname as rec_firstname
FROM Members as m1
INNER JOIN Members as m2
ON m2.memid = m1.recommendedby
WHERE m1.memid > 0
ORDER BY m1.surname, m1.firstname
        """
    cur.execute(query1)
    rows = cur.fetchall()
    for row in rows:
        print(row)

if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Bader', 'Florence', 'Stibbons', 'Ponder')
('Baker', 'Anne', 'Stibbons', 'Ponder')
('Baker', 'Timothy', 'Farrell', 'Jemima')
('Boothe', 'Tim', 'Rownam', 'Tim')
('Butters', 'Gerald', 'Smith', 'Darren')
('Coplin', 'Joan', 'Baker', 'Timothy')
('Crumpet', 'Erica', 'Smith', 'Tracy')
('Dare', 'Nancy', 'Joplette', 'Janice')
('Genting', 'Matthew', 'Butters', 'Gerald')
('Hunt', 'John', 'Purview', 'Millicent')
('Jones', 'David', 'Joplette', 'Janice')
('Jones', 'Douglas', 'Jones', 'David')
('Joplette', 'Janice', 'Smith', 'Darren')
('Mackenzie', 'Anna', 'Smith', 'Darren')
('Owen', 'Charles', 'Smith', 'Darren')
('Pinker', 'David', 'Farrell', 'Jemima')
('Purview', 'Millicent', 'Smith', 'Tracy')
('Rumney', 'Henrietta', 'Genting', 'Matthew')
('Sarwin', 'Ramnaresh', 'Bader', 'Florence')
('Smith', 'Jack', 'Smith', 'Darren')
('Stibbons', 'Ponder', 'Tracy', 'Burton')
('Worthington-Smyth', 'Henry', 'Smith', 'Tracy')


/* Q12: Find the facilities with their usage by member, but not guests */

In [11]:
# This returns the # of times each facility was used by members (i.e. summing all the slots)

def select_all_tasks(conn):
    cur = conn.cursor()
    query1 = """
SELECT
f.name as facility_name, 
SUM(b.slots) as fac_usage
FROM Bookings as b
LEFT JOIN Facilities as f
ON b.facid = f.facid
WHERE b.memid > 0
GROUP BY b.facid
        """
    cur.execute(query1)
    rows = cur.fetchall()
    for row in rows:
        print(row)

if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Tennis Court 1', 957)
('Tennis Court 2', 882)
('Badminton Court', 1086)
('Table Tennis', 794)
('Massage Room 1', 884)
('Massage Room 2', 54)
('Squash Court', 418)
('Snooker Table', 860)
('Pool Table', 856)


/* Q13: Find the facilities usage by month, but not guests */

In [14]:
# This returns the # of times each facility was used by members, per month (i.e. summing all the slots)

def select_all_tasks(conn):
    cur = conn.cursor()
    query1 = """
SELECT
f.name as facility_name, 
strftime('%m', starttime) as month,
SUM(b.slots) as fac_usage
FROM Bookings as b
LEFT JOIN Facilities as f
ON b.facid = f.facid
WHERE b.memid > 0
GROUP BY b.facid, month
        """
    cur.execute(query1)
    rows = cur.fetchall()
    for row in rows:
        print(row)

if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
('Tennis Court 1', '07', 201)
('Tennis Court 1', '08', 339)
('Tennis Court 1', '09', 417)
('Tennis Court 2', '07', 123)
('Tennis Court 2', '08', 345)
('Tennis Court 2', '09', 414)
('Badminton Court', '07', 165)
('Badminton Court', '08', 414)
('Badminton Court', '09', 507)
('Table Tennis', '07', 98)
('Table Tennis', '08', 296)
('Table Tennis', '09', 400)
('Massage Room 1', '07', 166)
('Massage Room 1', '08', 316)
('Massage Room 1', '09', 402)
('Massage Room 2', '07', 8)
('Massage Room 2', '08', 18)
('Massage Room 2', '09', 28)
('Squash Court', '07', 50)
('Squash Court', '08', 184)
('Squash Court', '09', 184)
('Snooker Table', '07', 140)
('Snooker Table', '08', 316)
('Snooker Table', '09', 404)
('Pool Table', '07', 110)
('Pool Table', '08', 303)
('Pool Table', '09', 443)
